<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/opeanai_aws_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [ ]:
!pip install --upgrade pip
!pip install colab-env --upgrade
!pip install openai
!pip install boto3
!pip install json



# AWS Credentials

In [2]:
import colab_env
import os

aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
region=os.getenv("region")
output=os.getenv("output")
FunctionName_ARN=os.getenv("AWS_LAMBDA_ARN")

Mounted at /content/gdrive


# Prompts and Completions: Definitions

In [3]:
import json
import openai # The OpenAI Python library provides convenient access to the OpenAI REST API.
import boto3  # The AWS SDK for Python (Boto3) provides a Python API for AWS infrastructure services.

## INVOKE AWS LAMBDA FUNCTION TO GET OPENAI API
def get_api_key():
    lambda_client = boto3.client('lambda')
    response = lambda_client.invoke(
            FunctionName = FunctionName_ARN,
            InvocationType = 'RequestResponse'
        )

    openai_api_key = json.load(response['Payload'])['body']['api_key']
    return openai_api_key


openai.api_key = get_api_key()

## GPT4
def lambda_handler_GPT4(prompt):
    stream = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )


    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            CC=chunk.choices[0].delta.content
            print(chunk.choices[0].delta.content, end="")
## GPT3.5
def lambda_handler_GPT3DOT5(prompt):
    model_to_use = "gpt-3.5-turbo-instruct"
    input_prompt=prompt

    openai.api_key = get_api_key()
    response = openai.completions.create(
      model=model_to_use,
      prompt=input_prompt,
      temperature=0.8,
      max_tokens=512,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )
    #print(response)
    return response
    #text_response = response['choices'][0]['text'].strip()


# Prompts and Completions: Executions

In [19]:

prompt = "what is the 40% of 30?"
prompt = "what is the 20.5% of 40?"
prompt = "what is the 30% of 650?"
prompt = "As a data scientist,can you explain the concept of regularization in machine learning?"
prompt="Write an email to Elon Musk asking him why he bought Twitter for such a huge amount"

print()
print('======================')
print()
prompt='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill.\n How many dollars did I get back? Explain first before answering.'
#prompt='Which country has the most natural lakes? Answer with only the country name.'
print('Prompt: %s'%prompt)

print()
print('======================')
print()
print()
print('Chat Completions - gpt4')
print()
print()
response=lambda_handler_GPT4(prompt)
print()
print('======================')
print()

print('Completions - gpt3.5')
response=lambda_handler_GPT3DOT5(prompt)
print()
#print('Answer:%s \r\n'%response.choices[0].text)
print('Answer (gpt 3.5):%s \n'%response.choices[0].text, sep='', flush=True)



Prompt: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill.
 How many dollars did I get back? Explain first before answering.



Chat Completions - gpt4


First, calculate the total cost of the ice cream cones by multiplying the cost of one ice cream cone by the total number of kids i.e. $1.25 * 6 kids = $7.5. This means that the total cost of ice cream for the six kids was $7.5. 

Then subtract the total cost of the ice cream from the amount paid i.e., $10 - $7.5 = $2.5. 

So, you got back $2.5.

Completions - gpt3.5

Answer (gpt 3.5):

To find out how much change you received, you need to subtract the total cost of the ice cream cones ($1.25 x 6 = $7.50) from the amount you paid with ($10).

$10 - $7.50 = $2.50

Therefore, you received $2.50 back as change. 

